In [1]:
import boto3

In [9]:
# bucket='sagemaker-pipeline-mnist-datasets' # Replace with your s3 bucket name
# prefix = 'source/tf-mnist' # Used as part of the path in the bucket where you store data

# train_prefix = 'train'
# val_prefix = 'validation'

# train_data = 's3://{}/{}/{}/'.format(bucket, project_name, train_prefix)
# validation_data = 's3://{}/{}/{}/'.format(bucket, project_name, val_prefix)

source_path = 's3://sagemaker-pipeline-mnist-datasets/source/tf-mnist/'
output_path = 's3://sagemaker-pipeline-mnist-datasets/output/tf-mnist/'
train_prefix = 'train'
val_prefix = 'validation'

In [ ]:

import sagemaker
from sagemaker.s3 import S3Uploader

session = sagemaker.Session()

glue_script_location = S3Uploader.upload(local_path='./train_val_norm.py',
                               desired_s3_uri='s3://sagemaker-pipeline-mnist-datasets/glue_script',
                               session=session)

job_name = 'train_val_norm_job'
glue_role = 'AWS-Glue-S3-Bucket-Access'

glue_client = boto3.client('glue')

response = glue_client.create_job(
    Name=job_name,
    Description='PySpark job to normalize the features of train and validation data',
    Role=glue_role, # you can pass your existing AWS Glue role here if you have used Glue before
    ExecutionProperty={
        'MaxConcurrentRuns': 2
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': glue_script_location,
        'PythonVersion': '3'
    },
    DefaultArguments={
        '--job-language': 'python'
    },
    GlueVersion='1.0',
    WorkerType='Standard',
    NumberOfWorkers=2,
    Timeout=60
)

In [ ]:
res = glue_client.start_job_run(
    JobName=job_name,
    Arguments={
        '--SOURCE_PATH': source_path,
        '--OUTPUT_PATH': output_path,
        '--TRAIN_PREFIX': train_prefix + '/',
        '--VAL_PREFIX': val_prefix + '/',
    },
)

In [8]:
print(res)

{'JobRunId': 'jr_b424c34b712cf32ae5762e195470185c182a51cb226073c3e6931a685fec5610', 'ResponseMetadata': {'RequestId': '2f97aaad-0020-4bb5-8b99-35f113852577', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 13 Apr 2020 09:34:37 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': '2f97aaad-0020-4bb5-8b99-35f113852577'}, 'RetryAttempts': 0}}
